In here we're going to explore a few different methods for extracting standard error from the component OPR model to see which ones hold up.

This is _essential_ to creating any kind of model that we'll use to get match score predictions, as we need to know what the match score confidence interval looks like to say by what margin a team will win by.

(I'd assume that in the future for that model we'll end up using some kind of "sliding confidence interval" to see at what point the intervals stop overlapping to determine a match victor)

(a lot of this is transcribed from some research/brainstorming i've been doing)

The main leads that I have for determining the error are:
- Using some kind of bootstrapping to simulate matches and find the standard error of the _overall_ OPR model
- Using the fact that $\frac{1}{n}\sqrt{\Sigma ^n _{i=0} (O_i - AP)^2} = SE$ to find the standard error for a _single team_ -- (O is observed match score, AP is match score prediction from model (incidence * oPr))
- Taking the above method and repeating it for all the teams, tuning the model being made as it goes, to eventaully get a better tuned model (and hope that it converges!) -- this seems a bit like an ML technique
- Looking at the mean of the standard error for the OPR fit/residuals when doing the matrix math in order to get a general uncertainty for each team
- A poor man's approach to calculating OPR (as outlined by [wgardner](https://www.chiefdelphi.com/t/standard-error-of-opr-values/144363/20)), which basically entails finding the variance and standard deviation of a team's OPR while omitting every match, one match at a time. This seems like less of a "standard error" metric, but still one that can be used to estimate match score variance

The first method that I'll be looking at is **wgardner's method**, as that seems like it'll be the simplest to implement.

In [53]:
from dataLoader import load_data_event

import numpy as np
import pandas as pd

import json
import math

In [9]:
# load in the data -- for this, we'll only be looking at data from one qualifier event to see if the method works
with open('data/week1/2020ncwak.json', 'r') as f:
    json_data = json.load(f)

team_keys = json_data['teams']

qualification_matches, _, _, _ = load_data_event('week1/2020ncwak')
qualification_matches

,match_key,match_type,match_number,blue_1_key,blue_2_key,blue_3_key,blue_keys,blue_endgame_level,blue_foul_count,blue_points_scored,...,red_3_init_line,red_1_endgame,red_2_endgame,red_3_endgame,red_cells_bottom_auto,red_cells_bottom_teleop,red_cells_outer_auto,red_cells_outer_teleop,red_cells_inner_auto,red_cells_inner_teleop
11,2020ncwak_qm1,qm,1,frc435,frc6565,frc5511,"[frc435, frc6565, frc5511]",True,1,81,...,5,0,5,5,0,0,0,0,0,0
12,2020ncwak_qm10,qm,10,frc5919,frc7265,frc4291,"[frc5919, frc7265, frc4291]",True,0,10,...,5,0,0,25,0,0,3,0,0,0
13,2020ncwak_qm11,qm,11,frc435,frc6240,frc6502,"[frc435, frc6240, frc6502]",False,0,73,...,5,5,5,5,0,9,0,0,0,0
14,2020ncwak_qm12,qm,12,frc3459,frc4828,frc5190,"[frc3459, frc4828, frc5190]",False,0,107,...,5,5,5,0,0,0,0,0,0,0
15,2020ncwak_qm13,qm,13,frc7890,frc5518,frc5511,"[frc7890, frc5518, frc5511]",True,0,92,...,5,5,5,0,0,4,0,1,0,0
16,2020ncwak_qm14,qm,14,frc5607,frc5160,frc2642,"[frc5607, frc5160, frc2642]",True,0,27,...,0,5,0,5,0,0,2,1,1,1
17,2020ncwak_qm15,qm,15,frc6500,frc3459,frc6496,"[frc6500, frc3459, frc6496]",False,1,59,...,5,5,5,5,0,0,3,3,0,0
18,2020ncwak_qm16,qm,16,frc2059,frc5919,frc5762,"[frc2059, frc5919, frc5762]",False,4,37,...,5,5,5,5,0,5,1,5,2,0
19,2020ncwak_qm17,qm,17,frc8090,frc7890,frc3229,"[frc8090, frc7890, frc3229]",True,0,50,...,5,5,5,5,0,0,0,6,0,0
20,2020ncwak_qm18,qm,18,frc7671,frc5160,frc7463,"[frc7671, frc5160, frc7463]",False,0,54,...,0,5,5,0,1,1,0,0,0,0


In [41]:
# common function for calculating a team's OPR given a point objective to get the OPR for and matches to look at
# still, thanks to [this](https://blog.thebluealliance.com/2017/10/05/the-math-behind-opr-an-introduction/)
def calculate_oprs(matches, team_keys, point_objective):
    match_matrix = np.zeros((len(matches)*2, len(team_keys)))
    score_matrix = np.zeros((len(matches)*2, 1))

    i = 0
    for _, match in matches.iterrows():
        for red_team in match['red_keys']:
            match_matrix[i*2][team_keys.index(red_team)] = 1
        score_matrix[i*2][0] = match[f'red_{point_objective}']

        for blue_team in match['blue_keys']:
            match_matrix[i*2+1][team_keys.index(blue_team)] = 1
        score_matrix[i*2+1][0] = match[f'blue_{point_objective}']

        i += 1

    l_matrix = match_matrix.T.dot(match_matrix)
    r_matrix = match_matrix.T.dot(score_matrix)
    opr_matrix = np.linalg.pinv(l_matrix).dot(r_matrix)

    return opr_matrix

In [45]:
# to implement wgardner's method, we'll need to iterate through teams and compute OPR while taking out matches that they're in
def wgardner_team(team, matches, team_keys, point_objective):
    # first get matches that the team is in {O(n)}
    matches_playing = []
    for i, match in matches.iterrows():
        if team in match['red_keys'] or team in match['blue_keys']:
            matches_playing.append(i)

    # iterate through the matches, calculate OPR w/out that match being considered
    oprs = []
    for match_removing in matches_playing:
        oprs.append(calculate_oprs(matches.drop(match_removing), team_keys, point_objective)[team_keys.index(team)][0])

    # take the mean and standard deviation
    std = np.std(oprs)
    mean = np.mean(oprs)

    return mean, std

In [61]:
# get a dataframe with the new OPR calculations
def get_opr_std_method(matches, team_keys, point_objective, opr_function):
    team_opr_data = []
    for team in team_keys:
        mean, std = opr_function(team, qualification_matches, team_keys, point_objective)
        team_opr_data.append([team, mean, std])
    return pd.DataFrame(team_opr_data, columns=['team_key', 'opr_mean', 'opr_std'])

opr_wgardner_data = get_opr_std_method(qualification_matches, team_keys, 'points_scored', wgardner_team)
opr_wgardner_data

,team_key,opr_mean,opr_std
0,frc2059,17.896851,2.428961
1,frc2642,22.791394,1.808675
2,frc3229,14.869575,0.864849
3,frc3459,36.204948,2.416582
4,frc4291,21.131777,2.353282
5,frc435,28.395997,2.545728
6,frc4561,33.783723,2.672946
7,frc4816,15.177462,1.763741
8,frc4828,30.759297,2.681994
9,frc5160,41.944778,1.908392


These results seem to be pretty similar between them and I'd say that they line up with what I'd expect the STD to look like.

I'll now be looking at the more mathematical approach of finding the residuals for all the matches that a team plays in and using that to compute standard error.

In [71]:
# for the math approach, we iterate through the matches that a team plays with initial OPR predictions, find the residual, and then use that in our error stuff
def residual_team(team, matches, team_keys, point_objective):
    # get initial OPR results
    opr_predictions_initial = calculate_oprs(qualification_matches, team_keys, point_objective)

    # go through and find residuals on a per match basis
    predictions = []
    targets = []
    for _, match in matches.iterrows():
        alliance_keys = observed_score = None
        if team in match['red_keys']:
            alliance_keys = match['red_keys']
            observed_score = match[f'red_{point_objective}']
        if team in match['blue_keys']:
            aliance_keys = match['blue_keys']
            observed_score = match[f'blue_{point_objective}']
        
        if alliance_keys != None:
            predictions.append(sum([
                opr_predictions_initial[team_keys.index(key)][0] for key in alliance_keys
            ]))
            targets.append(observed_score)
    predictions = np.array(predictions)
    targets = np.array(targets)

    # calculate std from residuals
    std = np.sqrt(np.mean((predictions-targets)**2))

    return opr_predictions_initial[team_keys.index(team)][0], std

In [73]:
opr_residual_data = get_opr_std_method(qualification_matches, team_keys, 'points_scored', residual_team)
opr_residual_data

,team_key,opr_mean,opr_std
0,frc2059,17.894015,29.535132
1,frc2642,22.768461,10.053786
2,frc3229,14.960390,9.729637
3,frc3459,36.080602,19.882051
4,frc4291,21.166143,23.294821
5,frc435,28.398827,30.158297
6,frc4561,33.786561,20.391414
7,frc4816,15.268651,18.272789
8,frc4828,30.687604,22.782791
9,frc5160,42.045429,10.967039


With this approach it looks like we get a bit (lit. a lot) more error than before, which could also make sense given that OPR is potentially bogus as a scoring metric.

Looking at the two metrics that I've implemented, I think that they both serve their own purposes. wgardner's method is more suited to looking at the variance of the OPR metric itself, while the residual method is more suited for looking at a team's performance while taking OPR into consideration. Between the two, I think that we'll likely get better results when using the residual method, since our models are mainly built off of the prediction of the metric.